In [3]:
import sys
import pandas as pd

# Alternate between ../../ and ../../src due to bug
sys.path.append('../../src')

from src.cryptolib.enums import ExchangeType
from src.service.exchange import ExchangeService
from src.config import config
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

ENGINE = create_engine(config.SQLALCHEMY_DATABASE_URI, echo=False, pool_size=20, max_overflow=0)

exchange = ExchangeService().get_exchange(ExchangeType.BINANCE.value, config.BINANCE_API_KEY, config.BINANCE_API_SECRET)

# Checking if the values are the same
print(exchange.get_last_price('ETHUSDT'))
print('Sandbox:', exchange.sandbox)

1814.71
Sandbox: False


In [4]:
# markets = prod.load_markets()
df = pd.DataFrame(exchange.get_all_symbols()).T
df.head()

,symbol,timestamp,datetime,high,low,bid,bidVolume,ask,askVolume,vwap,open,close,last,previousClose,change,percentage,average,baseVolume,quoteVolume,info
ETH/BTC,ETH/BTC,1684664559738,2023-05-21T10:22:39.738Z,0.0676,0.06685,0.06707,15.4416,0.06708,22.9132,0.06719,0.06746,0.06708,0.06708,0.06746,-0.00038,-0.563,0.06727,16878.9654,1134.09151,"{'symbol': 'ETHBTC', 'priceChange': '-0.000380..."
LTC/BTC,LTC/BTC,1684664558269,2023-05-21T10:22:38.269Z,0.003456,0.003382,0.003434,84.655,0.003436,93.759,0.003414,0.003392,0.003435,0.003435,0.003393,0.000043,1.268,0.003413,49685.248,169.635739,"{'symbol': 'LTCBTC', 'priceChange': '0.0000430..."
BNB/BTC,BNB/BTC,1684664560090,2023-05-21T10:22:40.090Z,0.011522,0.011416,0.011494,5.989,0.011495,96.673,0.011479,0.011487,0.011495,0.011495,0.011488,0.000008,0.07,0.011491,13229.889,151.869257,"{'symbol': 'BNBBTC', 'priceChange': '0.0000080..."
NEO/BTC,NEO/BTC,1684664554443,2023-05-21T10:22:34.443Z,0.00035,0.000345,0.000347,22.09,0.000348,29.32,0.000348,0.00035,0.000347,0.000347,0.000349,-0.000002,-0.658,0.000349,8755.23,3.045598,"{'symbol': 'NEOBTC', 'priceChange': '-0.000002..."
QTUM/ETH,QTUM/ETH,1684664091125,2023-05-21T10:14:51.125Z,0.001457,0.001441,0.001439,38.2,0.001448,200.0,0.001447,0.001456,0.001441,0.001441,0.00145,-0.000015,-1.03,0.001448,132.4,0.191645,"{'symbol': 'QTUMETH', 'priceChange': '-0.00001..."


### Generating load test data

##### Users
Creates N users.

In [21]:
import random
import string

from src.cryptolib.model import UserModel

def randomString(stringLength=10) -> str:
    letters = string.ascii_letters
    return ''.join([random.choice(letters) for i in range(stringLength)])

number_of_users = 1000

with Session(ENGINE) as session:
    for i in range(number_of_users):
        rnd = randomString()
        user = UserModel(
            email=rnd + '@gmail.com',
            password=rnd,
        )
        session.add(user)
        session.flush()

    # session.commit()

##### Symbols & Currency pairs

In [11]:
target_currency = "USDT"

existing_symbols = set()
invalid_symbols = {'YOYOW/ETH', 'YOYOW/BNB', 'YOYOW/BTC', 'BTCDOM/USDT', 'LUNA2/USDT', 'LUNA/USDT', 'BLUEBIRD/USDT', 'DEFI/USDT', '1000SHIB/USDT', 'MANA/USDT', '1000LUNC/USDT', 'BLUR/USDT', 'FOOTBALL/USDT', '1000PEPE/USDT', '1000FLOKI/USDT', '1000XEC/USDT', 'NBTUSDT'}

avoid = existing_symbols.union(invalid_symbols)

# Get all the symbols
currency_pairs = set(df["symbol"].to_list())
currency_pairs = [symbol for symbol in currency_pairs if symbol.endswith(target_currency) and symbol not in avoid]
currency_pairs[:5]

['RVN/USDT', 'BTCDOWN/USDT', 'WAVES/USDT', 'OM/USDT', 'SNT/USDT']

In [27]:
symbol_query = "INSERT INTO symbols (symbol) VALUES ('{}');"
symbol_queries = [symbol_query.format(currency_pair.split('/')[0]) for currency_pair in currency_pairs]

currency_pair_queries = []
for currency_pair in currency_pairs:
    currency_pair_clean = currency_pair.replace('/', '')

    try:
        symbol = currency_pair.split('/')[0]
        pair = currency_pair.split('/')[1]
        currency_pair_queries.append(f"INSERT INTO currency_pairs (symbol, pair, currency_pair) VALUES ('{symbol}', '{pair}', '{currency_pair_clean}');")
    except:
        print(currency_pair)

with open('symbols.load.sql', 'w+') as f:
    for query in symbol_queries:
        f.write(query + '\n')
    
    for query in currency_pair_queries:
        f.write(query + '\n')

## Bot generation

In [5]:
import sys
sys.path.append('../../')

from src.config import config

from src.cryptolib.enums import StrategyType, ExchangeType, Interval
from src.cryptolib.model import UserModel, ApiKeyModel, PortfolioModel, BalanceModel, CurrencyPairConfigModel, SignalModel, StrategyConfigModel

print('Sandbox:', config.API_SANDBOX)

Sandbox: True


In [6]:
# Queries
user_query = "INSERT INTO users (id, email, _password) VALUES ('{}', '{}', '{}');"

api_key_query = "INSERT INTO api_keys (user_id, exchange, api_key, api_secret) VALUES ('{}', '{}', '{}', '{}');"

portfolio_query = "INSERT INTO portfolios (id, user_id) VALUES ({}, '{}');"

balance_query = "INSERT INTO balances (portfolio_id, asset, free, locked, total) VALUES ('{}', '{}', {}, {}, {});"

currency_pair_config_query = "INSERT INTO currency_pair_configs (user_id, currency_pair, bot_name, exchange, `interval`, strategy, `limit`, is_active, stop_loss, take_profit, allocated_balance, currency_free, currency_locked, asset_free, asset_locked) VALUES ('{}', '{}', '{}', '{}', '{}', '{}', 1000, 1, 0.3, 0.3, {}, {}, 0.0, 0.0, 0.0);"

signal_query = "INSERT INTO signals (currency_pair_config_id, `signal`) VALUES ({}, 'HOLD');"

strategy_config_query = "INSERT INTO strategy_configs (currency_pair_config_id, strategy, `key`, `value`) VALUES ({}, '{}', '{}', {});"

In [7]:
bot_config = {
    'email': 'bot{}@admin.com',
    'password': '$2b$12$F/EmqrDOVllQXDkO9xBXbexbwXVit/GuXOxSvAiQ3NrvPSwJCv7VK',
    'name': 'Bot {} - {}',
    'starting_currency': 5000.0,
    # 'symbols': ['MINA', 'AXS', 'TRU', 'CKB', 'EDU'], # Great for RSI 1m
    'symbols': ['XRP'],
    'currency': 'USDT',
    'api_key': config.BINANCE_API_KEY,
    'api_secret': config.BINANCE_API_SECRET,
    'exchange': ExchangeType.BINANCE.name
}

strategies = [
    {
        'strategy': StrategyType.RSI.name,
        'params': {'window': 1},
        'interval': Interval.MINUTE_1.name
    },
]

In [8]:
# Create user
user_queries = []
# Create api key
api_key_queries = []
# Create portfolio
portfolio_queries = []
# Create balances
balance_queries = []
# Create currency pair config
currency_pair_config_queries = []
# Create signal
signal_queries = []
# Create strategy config
strategy_config_queries = []

starting_user_id = 48 # starting user id
config_id = 945 # starting config id
for i, strategy in enumerate(strategies):
    id = starting_user_id + i
    strategy_type = strategy['strategy']
    params = strategy['params']
    interval = strategy['interval']

    # Create user
    query = user_query.format(id, bot_config['email'].format(id), bot_config['password'])
    user_queries.append(query)

    # Create api key
    query = api_key_query.format(id, bot_config['exchange'], bot_config['api_key'], bot_config['api_secret'])
    api_key_queries.append(query)

    # Create portfolio
    query = portfolio_query.format(id, id)
    portfolio_queries.append(query)

    # Create balances
    query = balance_query.format(
        id,
        'USDT',
        0.0,
        bot_config['starting_currency'] * len(bot_config['symbols']),
        bot_config['starting_currency'] * len(bot_config['symbols'])
    )
    balance_queries.append(query)
    for symbol in bot_config['symbols']:
        query = balance_query.format(id, symbol, 0.0, 0.0, 0.0)
        balance_queries.append(query)

        # Create currency pair config
        query = currency_pair_config_query.format(
            id,
            f"{symbol}{bot_config['currency']}",
            bot_config['name'].format(id, symbol),
            bot_config['exchange'],
            interval,
            strategy_type,
            bot_config['starting_currency'],
            bot_config['starting_currency']
        )
        currency_pair_config_queries.append(query)
    
        # Create signal
        query = signal_query.format(config_id)
        signal_queries.append(query)

        # Create strategy config
        for key, value in params.items():
            query = strategy_config_query.format(config_id, strategy_type, key, value)
            strategy_config_queries.append(query)
        
        config_id += 1
        

with open('bot.testing.sql', 'w+') as f:
    for query in user_queries:
        f.write(query + '\n')
    
    for query in api_key_queries:
        f.write(query + '\n')
    
    for query in portfolio_queries:
        f.write(query + '\n')
    
    for query in balance_queries:
        f.write(query + '\n')
    
    for query in currency_pair_config_queries:
        f.write(query + '\n')
    
    for query in signal_queries:
        f.write(query + '\n')
    
    for query in strategy_config_queries:
        f.write(query + '\n')

## SQL Debugging and fixing

In [28]:
from src.service import DataService
from sqlalchemy import text

##### Updating invalid strategy in the strategy configs

In [29]:
with Session(ENGINE) as session:
    bots = DataService().get_bots(session, '45')
    for bot in bots:
        configs = bot.strategy_config
        for config in configs:
            config.strategy = StrategyType.CHAIKIN_OSCILLATOR.name
            # session.flush()

    # session.commit()

##### Deleting invalid symbols

In [21]:
with ENGINE.connect() as conn:
    for currency_pair in invalid_symbols:
        symbol = currency_pair[:-4]
        conn.execute(text(f"DELETE FROM symbols WHERE symbol = '{symbol}'"))
        conn.execute(text(f"DELETE FROM currency_pair_configs WHERE currency_pair = '{currency_pair}'"))
        conn.commit()